# Executive Compensation Analysis

Deep dive into Fortune 100 executive compensation data extracted from DEF 14A filings.

**Contents:**
1. Load compensation data
2. CEO compensation rankings
3. Compensation trends by year
4. Sector analysis
5. Component breakdown (salary, stock, bonus)

In [ ]:
# Setup
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import pandas as pd
import json

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.0f}')

## 1. Load Compensation Data

In [ ]:
# Load the executive compensation CSV
output_dir = Path.cwd().parent / "output" / "fortune100"
comp_df = pd.read_csv(output_dir / "executive_compensation.csv")

print(f"Records: {len(comp_df):,}")
print(f"Companies: {comp_df['company'].nunique()}")
print(f"Executives: {comp_df['executive_name'].nunique()}")
print(f"\nFiscal Years: {sorted(comp_df['fiscal_year'].unique())}")

In [ ]:
# Preview data structure
comp_df.head(10)

## 2. CEO Compensation Rankings

In [ ]:
# Filter to CEOs only (most recent year)
latest_year = comp_df['fiscal_year'].max()

ceo_df = comp_df[
    (comp_df['title'].str.contains('CEO|Chief Executive', case=False, na=False)) &
    (comp_df['fiscal_year'] == latest_year)
].copy()

# Sort by total compensation
ceo_df = ceo_df.sort_values('total_comp', ascending=False)

print(f"\nTop 10 Highest Paid CEOs (FY {latest_year}):\n")
print(ceo_df[['rank', 'company', 'executive_name', 'total_comp']].head(10).to_string(index=False))

In [ ]:
# CEO compensation statistics
print(f"\nCEO Compensation Statistics (FY {latest_year}):")
print(f"  Mean: ${ceo_df['total_comp'].mean():,.0f}")
print(f"  Median: ${ceo_df['total_comp'].median():,.0f}")
print(f"  Max: ${ceo_df['total_comp'].max():,.0f}")
print(f"  Min: ${ceo_df['total_comp'].min():,.0f}")

## 3. Compensation Trends by Year

In [ ]:
# Average compensation by year
yearly_avg = comp_df.groupby('fiscal_year')['total_comp'].agg(['mean', 'median', 'count'])
yearly_avg.columns = ['Mean', 'Median', 'Count']

print("Average Executive Compensation by Year:\n")
print(yearly_avg.to_string())

In [ ]:
# Year-over-year change for top companies
top_companies = comp_df.groupby('company')['total_comp'].sum().nlargest(5).index

for company in top_companies:
    company_df = comp_df[comp_df['company'] == company]
    yearly_total = company_df.groupby('fiscal_year')['total_comp'].sum()
    print(f"\n{company}:")
    for year, total in yearly_total.items():
        print(f"  FY {year}: ${total:,.0f}")

## 4. Sector Analysis

In [ ]:
# Load Fortune 100 registry for sector data
from edgar.data.fortune100 import Fortune100Registry

registry = Fortune100Registry.load_default()

# Create ticker-to-sector mapping
sector_map = {c.ticker: c.sector for c in registry.companies}

# Add sector to dataframe
comp_df['sector'] = comp_df['ticker'].map(sector_map)

# Average compensation by sector
sector_avg = comp_df.groupby('sector')['total_comp'].agg(['mean', 'median', 'count'])
sector_avg = sector_avg.sort_values('mean', ascending=False)
sector_avg.columns = ['Mean', 'Median', 'Count']

print("Average Executive Compensation by Sector:\n")
print(sector_avg.to_string())

## 5. Compensation Component Breakdown

In [ ]:
# Component breakdown for latest year
latest_df = comp_df[comp_df['fiscal_year'] == latest_year].copy()

components = ['salary', 'bonus', 'stock_awards', 'option_awards', 'non_equity_incentive', 'other_comp']

print(f"Compensation Components (FY {latest_year}):\n")

for comp in components:
    if comp in latest_df.columns:
        total = latest_df[comp].sum()
        avg = latest_df[comp].mean()
        pct = (total / latest_df['total_comp'].sum()) * 100
        print(f"  {comp:25s}: Total ${total:>15,.0f} | Avg ${avg:>12,.0f} | {pct:5.1f}%")

In [ ]:
# Top executives by stock awards
if 'stock_awards' in latest_df.columns:
    stock_leaders = latest_df.nlargest(10, 'stock_awards')[['company', 'executive_name', 'stock_awards', 'total_comp']]
    print(f"\nTop 10 Stock Award Recipients (FY {latest_year}):\n")
    print(stock_leaders.to_string(index=False))

## Summary

Key findings from executive compensation analysis:

1. **CEO Pay**: Highest paid CEOs and compensation statistics
2. **Trends**: Year-over-year changes in compensation
3. **Sectors**: Which industries pay executives the most
4. **Components**: How compensation breaks down (salary vs stock vs bonus)

Next: **03_tax_analysis.ipynb** for corporate tax analysis